In [19]:
import datafetching, datetime, json
import pandas as pd
from google.cloud import storage

In [20]:
import os
os.environ.setdefault("GCLOUD_PROJECT", "disco-sky-405321")

'disco-sky-405321'

In [21]:
# GCS
client = storage.Client()
bucket = client.get_bucket('gw2pricedata-dump')
item_description_location = "items/item_description/"
blob = bucket.blob(item_description_location + "item_description2023-11-18 00_45_27")

In [22]:
data = json.loads(blob.download_as_string(client=None))

In [23]:
item_desc_df = pd.DataFrame(data["data"])

## Find items in relevant categories
Don't need to fetch prices for every item in the game, only those I'm interested in training on.
Possible categories: Armor, Back, Bag, Consumable, Container, CraftingMaterial, Gathering,MiniPet, PowerCore, Trinket, UpgradeComponent, Weapon

Should also filter out items flagged as being accountbound or soulbound
Flags: AccountBound, NoSell, SoulbindOnAcquire

In [24]:
filter_condition = lambda flags: all(item not in flags for item in ["SoulbindOnAcquire", "AccountBound", "NoSell"])
filtered_df = item_desc_df[item_desc_df['flags'].apply(filter_condition)]

In [25]:
categories = ["Armor", "Back", "Bag", "Consumable", "Container", 
              "CraftingMaterial", "Gathering", "MiniPet", "PowerCore", 
              "Trinket", "UpgradeComponent", "Weapon"]

# Filter the DataFrame
filtered_categories_df = filtered_df[filtered_df['type'].isin(categories)]

In [26]:
filtered_categories_df.head()

,name,type,level,rarity,vendor_value,default_skin,game_types,flags,restrictions,id,chat_link,icon,details,description,upgrades_into,upgrades_from
3,Sealed Package of Snowballs,Consumable,0,Basic,0,NaN,"[Wvw, Pve]",[NoSalvage],[],24,[&AgEYAAAA],https://render.guildwars2.com/file/1D05D1EE04E...,{'type': 'Generic'},Open this package to create several snowballs ...,NaN,NaN
5,Gladiator Weapon,Weapon,0,Basic,0,3265.0,"[Activity, Wvw, Dungeon, Pve]",[NoSalvage],[],46,[&AgEuAAAA],https://render.guildwars2.com/file/4AECE5EA59C...,"{'type': 'Staff', 'damage_type': 'Physical', '...",NaN,NaN,NaN
16,Mighty Country Coat,Armor,0,Basic,6,13.0,"[Activity, Wvw, Dungeon, Pve]",[NotUpgradeable],[],68,[&AgFEAAAA],https://render.guildwars2.com/file/F03808FFE89...,"{'type': 'Coat', 'weight_class': 'Light', 'def...",,NaN,NaN
17,Mighty Country Coat,Armor,0,Basic,7,13.0,"[Activity, Wvw, Dungeon, Pve]",[NotUpgradeable],[],69,[&AgFFAAAA],https://render.guildwars2.com/file/F03808FFE89...,"{'type': 'Coat', 'weight_class': 'Light', 'def...",,NaN,NaN
18,Mighty Studded Coat,Armor,0,Basic,6,17.0,"[Activity, Wvw, Dungeon, Pve]",[NotUpgradeable],[],70,[&AgFGAAAA],https://render.guildwars2.com/file/5050F9A0AAA...,"{'type': 'Coat', 'weight_class': 'Medium', 'de...",NaN,NaN,NaN


## Split items into categories
Then upload to corresponding category folder in blob storage

In [27]:
consumables = filtered_categories_df[filtered_categories_df["type"] == "Consumable"]
consumables_ids = consumables["id"].tolist()

In [28]:
for item in consumables_ids:
    item_data = datafetching.get_item_history(item)
    jsonItems = {"data": item_data}
    bucket.blob('items/consumable/' + str(item) + " " + datetime.datetime.now().strftime('%Y-%m-%d%H_%M_%S')).upload_from_string(data=json.dumps(jsonItems),content_type='application/json')

https://api.datawars2.ie/gw2/v2/history/json?itemID=24
https://api.datawars2.ie/gw2/v2/history/json?itemID=8495
https://api.datawars2.ie/gw2/v2/history/json?itemID=8511
https://api.datawars2.ie/gw2/v2/history/json?itemID=8515
https://api.datawars2.ie/gw2/v2/history/json?itemID=8535
https://api.datawars2.ie/gw2/v2/history/json?itemID=8553
https://api.datawars2.ie/gw2/v2/history/json?itemID=8568
https://api.datawars2.ie/gw2/v2/history/json?itemID=8574
https://api.datawars2.ie/gw2/v2/history/json?itemID=8576
https://api.datawars2.ie/gw2/v2/history/json?itemID=8578
https://api.datawars2.ie/gw2/v2/history/json?itemID=8579
https://api.datawars2.ie/gw2/v2/history/json?itemID=8583
https://api.datawars2.ie/gw2/v2/history/json?itemID=8591
https://api.datawars2.ie/gw2/v2/history/json?itemID=8592
https://api.datawars2.ie/gw2/v2/history/json?itemID=8593
https://api.datawars2.ie/gw2/v2/history/json?itemID=8594
https://api.datawars2.ie/gw2/v2/history/json?itemID=8600
https://api.datawars2.ie/gw2/v2/h